In [3]:
import numpy as np
from numpy.linalg import inv
from scipy.stats import invgamma, bernoulli, multivariate_normal
from scipy.stats import beta as beta_dist  # 修正之前错误，将beta的定义从numpy切换到scipy.stats

# 生成数据的函数
def generate_data(n, p_f, p, q, true_params):
    """
    根据真实参数生成数据X_f, X, Z和响应变量y。
    参数：
    - n: 样本数
    - p_f: 固定效应数量
    - p: 稀疏固定效应数量
    - q: 随机效应数量
    - true_params: 真实参数的字典

    返回值：
    - X_f, X, Z: 模型设计矩阵
    - y: 响应变量
    """
    # 生成设计矩阵 X_f, X, Z
    X_f = np.random.normal(0, 1, size=(n, p_f))
    X = np.random.normal(0, 1, size=(n, p))
    Z = np.random.normal(0, 1, size=(n, q))

    # 提取真实参数
    beta_f_true = true_params['beta_f']
    beta_true = true_params['beta']
    u_true = true_params['u']
    sigma2_true = true_params['sigma2']

    # 生成噪声
    epsilon = np.random.normal(0, np.sqrt(sigma2_true), size=n)

    # 根据模型生成响应变量 y
    y = X_f @ beta_f_true + X @ beta_true + Z @ u_true + epsilon

    return X_f, X, Z, y


# 修正Gibbs采样中的错误定义
def BSLMM_Gibbs_Sampler(y, X_f, X, Z, G, n_iter=1000):
    # 初始化参数
    n, p_f = X_f.shape
    _, p = X.shape
    _, q = Z.shape

    # 超参数设置（可根据需要调整）
    a_sigma = b_sigma = 1e-3
    a_u = b_u = 1e-3
    a_beta = b_beta = 1e-3
    a_beta_f = b_beta_f = 1e-3
    a_pi = b_pi = 1
    sigma_beta2 = 1.0
    sigma_beta_f2 = 1.0
    sigma_u2 = 1.0
    sigma2 = 1.0
    pi = 0.1

    # 初始化变量
    beta_f = np.zeros(p_f)
    beta = np.zeros(p)
    delta = np.zeros(p)
    u = np.zeros(q)

    # 存储结果
    samples = {
        'beta_f': [],
        'beta': [],
        'delta': [],
        'pi': [],
        'u': [],
        'sigma2': [],
        'sigma_u2': [],
        'sigma_beta2': [],
        'sigma_beta_f2': []
    }

    for it in range(n_iter):
        # 更新 beta_f
        X_fT_X_f = X_f.T @ X_f
        Sigma_beta_f = inv((X_fT_X_f / sigma2) + (np.eye(p_f) / sigma_beta_f2))
        y_prime = y - X @ beta - Z @ u
        mu_beta_f = Sigma_beta_f @ (X_f.T @ y_prime / sigma2)
        beta_f = multivariate_normal.rvs(mean=mu_beta_f, cov=Sigma_beta_f)

        # 更新 beta 和 delta
        for j in range(p):
            # 计算 delta_j 的后验概率
            beta_j = beta[j]
            X_j = X[:, j]
            y_j = y - X_f @ beta_f - X @ beta + X_j * beta_j - Z @ u
            # 计算似然比
            p_delta1 = pi
            p_delta0 = 1 - pi
            delta_j_prob = p_delta1 / (p_delta1 + p_delta0)
            delta_j = bernoulli.rvs(delta_j_prob)
            delta[j] = delta_j
            if delta_j == 1:
                # 更新 beta_j
                Sigma_beta_j = 1 / ((X_j.T @ X_j) / sigma2 + 1 / sigma_beta2)
                mu_beta_j = Sigma_beta_j * (X_j.T @ y_j / sigma2)
                beta[j] = np.random.normal(mu_beta_j, np.sqrt(Sigma_beta_j))
            else:
                beta[j] = 0.0

        # 更新 pi
        s = np.sum(delta)
        pi = beta_dist.rvs(a_pi + s, b_pi + p - s)

        # 更新 u
        ZT_Z = Z.T @ Z
        Sigma_u = inv((ZT_Z / sigma2) + inv(sigma_u2 * G))
        y_u = y - X_f @ beta_f - X @ beta
        mu_u = Sigma_u @ (Z.T @ y_u / sigma2)
        u = multivariate_normal.rvs(mean=mu_u, cov=Sigma_u)

        # 更新 sigma2
        e = y - X_f @ beta_f - X @ beta - Z @ u
        a_post = a_sigma + n / 2
        b_post = b_sigma + (e.T @ e) / 2
        sigma2 = invgamma.rvs(a=a_post, scale=b_post)

        # 更新 sigma_u2
        a_post = a_u + q / 2
        b_post = b_u + (u.T @ inv(G) @ u) / 2
        sigma_u2 = invgamma.rvs(a=a_post, scale=b_post)

        # 更新 sigma_beta2
        s = np.sum(delta)
        beta_nonzero = beta[delta == 1]
        a_post = a_beta + s / 2
        b_post = b_beta + (beta_nonzero @ beta_nonzero) / 2
        sigma_beta2 = invgamma.rvs(a=a_post, scale=b_post)

        # 更新 sigma_beta_f2
        a_post = a_beta_f + p_f / 2
        b_post = b_beta_f + (beta_f.T @ beta_f) / 2
        sigma_beta_f2 = invgamma.rvs(a=a_post, scale=b_post)

        # 存储结果
        samples['beta_f'].append(beta_f.copy())
        samples['beta'].append(beta.copy())
        samples['delta'].append(delta.copy())
        samples['pi'].append(pi)
        samples['u'].append(u.copy())
        samples['sigma2'].append(sigma2)
        samples['sigma_u2'].append(sigma_u2)
        samples['sigma_beta2'].append(sigma_beta2)
        samples['sigma_beta_f2'].append(sigma_beta_f2)

    return samples


In [5]:
# 真实参数设定
n = 100   # 样本数
p_f = 5   # 固定效应数量
p = 10    # 稀疏固定效应数量
q = 3     # 随机效应数量

true_params = {
    'beta_f': np.random.normal(0, 1, size=p_f),
    'beta': np.concatenate([np.random.normal(0, 1, size=3), np.zeros(p - 3)]),
    'u': np.random.normal(0, 1, size=q),
    'sigma2': 1.0
}

# 生成数据
X_f, X, Z, y = generate_data(n, p_f, p, q, true_params)

# 基因关系矩阵 G
G = np.eye(q)
# 使用Gibbs采样算法估计参数
n_iter = 10000
samples = BSLMM_Gibbs_Sampler(y, X_f, X, Z, G, n_iter=n_iter)

# 估计结果与真实参数比较
estimated_beta_f = np.mean(samples['beta_f'], axis=0)
estimated_beta = np.mean(samples['beta'], axis=0)
estimated_u = np.mean(samples['u'], axis=0)
estimated_sigma2 = np.mean(samples['sigma2'])

estimated_beta_f, estimated_beta, estimated_u, estimated_sigma2, true_params


(array([-0.8554969 ,  1.24252586, -0.43752624, -1.14415925, -1.43525275]),
 array([ 1.06029806e+00, -2.27152166e-01, -1.65260440e-01, -1.06255027e-01,
         5.40153994e-02, -3.58327242e-02,  8.61252262e-04, -1.53241402e-02,
         6.79939761e-02, -1.90855554e-02]),
 array([-0.61831603, -0.33091535, -1.55483568]),
 6.130583168559898,
 {'beta_f': array([-0.67696576,  1.32426065, -0.51676523, -1.00561541, -1.44221155]),
  'beta': array([ 2.73375883, -0.61721936, -0.53163061,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ]),
  'u': array([-0.96962137, -0.13561451, -1.63013414]),
  'sigma2': 1.0})